In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizer
import os
import pandas as pd
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch.optim as optim
from collections import OrderedDict
import torch
from collections import OrderedDict
from torch.optim import Adam
import numpy as np
import torch
from pkg_resources import packaging
import torch.nn.functional as F
from pkg_resources import packaging
import clip

/home/ajeet/miniconda3/envs/distributedsystems/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2764575/3817544335.py:22: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import packaging


In [5]:
!git clone https://github.com/ttumyche/AI504_Practice14.git
!mv AI504_Practice14/simple_tokenizer.py /content/
!mv AI504_Practice14/bpe_simple_vocab_16e6.txt.gz /content/

Cloning into 'AI504_Practice14'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 1.30 MiB | 4.70 MiB/s, done.
mv: cannot move 'AI504_Practice14/simple_tokenizer.py' to '/content/': Not a directory
mv: cannot move 'AI504_Practice14/bpe_simple_vocab_16e6.txt.gz' to '/content/': Not a directory


In [2]:
from AI504_Practice14.simple_tokenizer import SimpleTokenizer
def tokenize_text(tokenizer, text, max_length):
    tokens = tokenizer.encode(text)
    if len(tokens) == 0:
        print(f"Empty tokens for text: {text}")
        return torch.tensor([], dtype=torch.long)

    tokens = tokens[:max_length]
    if len(tokens) < max_length:
        tokens.extend([0] * (max_length - len(tokens)))

    return torch.tensor(tokens, dtype=torch.long)


In [12]:
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

In [15]:
!mkdir datasets

In [25]:
!rm -f Flicker8k_Dataset

rm: cannot remove 'Flicker8k_Dataset': Is a directory


In [31]:
!unzip datasets/Flickr8k_Dataset.zip

Archive:  datasets/Flickr8k_Dataset.zip
replace Flicker8k_Dataset/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [32]:
!unzip datasets/Flickr8k_text.zip

Archive:  datasets/Flickr8k_text.zip
  inflating: CrowdFlowerAnnotations.txt  
  inflating: ExpertAnnotations.txt   
  inflating: Flickr8k.lemma.token.txt  
   creating: __MACOSX/
  inflating: __MACOSX/._Flickr8k.lemma.token.txt  
  inflating: Flickr8k.token.txt      
  inflating: Flickr_8k.devImages.txt  
  inflating: Flickr_8k.testImages.txt  
  inflating: Flickr_8k.trainImages.txt  
  inflating: readme.txt              


In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch

from PIL import Image
import os
import pandas as pd



class Flickr8kDataset(Dataset):
    def __init__(self, image_dir, captions_file, split_file, transform=None, tokenizer=None, max_length=77):
        # Load and prepare data
        self.image_dir = image_dir
        self.df = pd.read_csv(captions_file, sep='\t', names=['image', 'caption'])
        self.split_file = pd.read_csv(split_file, names=['image'])

        # Filter dataset based on split
        self.df['image'] = self.df['image'].apply(lambda x: x.split('#')[0])
        self.df = self.df[self.df['image'].isin(self.split_file['image'])]

        # Remove empty captions
        self.df = self.df[self.df['caption'].str.strip() != '']

        self.transform = transform
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        caption = self.df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        if self.tokenizer:
            caption = tokenize_text(self.tokenizer, caption, self.max_length)
            if len(caption) == 0:
                print(f"Empty caption at index {idx}: {self.df.iloc[idx, 1]}")
        else:
            caption = torch.tensor([], dtype=torch.long)

        return image, caption
# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

In [5]:
import clip
tokenizer = clip.tokenize

In [6]:
train_dataset = Flickr8kDataset(
    image_dir='Flicker8k_Dataset',
    captions_file='Flickr8k.token.txt',
    split_file='Flickr_8k.trainImages.txt',
    transform=transform,
    tokenizer=tokenizer,
    max_length=77
)

dev_dataset = Flickr8kDataset(
    image_dir='Flicker8k_Dataset',
    captions_file='Flickr8k.token.txt',
    split_file='Flickr_8k.devImages.txt',
    tokenizer=tokenizer,
    max_length=77
)

test_dataset = Flickr8kDataset(
    image_dir='Flicker8k_Dataset',
    captions_file='Flickr8k.token.txt',
    split_file='Flickr_8k.testImages.txt',
    tokenizer=tokenizer,
    max_length=77
)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [24]:
from src.CLIP import CLIP
from src.Training import Trainer

In [25]:
img_res = 224
vision_patch = 16
vision_width = 768
vision_layers = 4
vision_heads = 12
embed_dim = 128
context_length = 77
vocab_size = 49408
trans_width = 512
trans_heads = 8
trans_layers = 6

model = CLIP(
    embed_dim=512,
    img_res=224,
    vision_layers=12,
    vision_width=768,
    vision_patch=16,
    vision_heads=8,
    context_length=77,
    vocab_size=49408,
    trans_width=512,
    trans_heads=8,
    trans_layers=6
)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

NameError: name 'ImageEncoder' is not defined